In [ ]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df_dbv = pd.read_csv('/content/DBV Cleaned Data.csv')
df_trends = pd.read_excel('/content/World_GT_Full.xlsx')

In [ ]:
df_trends['Date'] = pd.to_datetime(df_trends.iloc[:, 0], infer_datetime_format=True)
df_dbv['Date'] = pd.to_datetime(df_dbv.iloc[:, 0], infer_datetime_format=True)

In [ ]:
ts_df = df_dbv.merge(df_trends, how = 'inner', on = ['Date'])

In [ ]:
ts_df.iloc[:,3:] = ts_df.iloc[:,3:].diff()

In [ ]:
ts_df

,Date,Adj Close,Log Difference,swiss franc,Europe,EUR,Switzerland,CHF,NASDAQ,S&P500,VIX,volatility,Germany,UK,sterling pound,GBP,Greece,Italy,Spain,France,DAX,unemployment,recession,euro,crisis,fed,finance,financial crisis,federal reserve,gold,inflation,treasuries,OPEC,contagion,corruption,disaster,futures,forwards,president,parliament,...,steel,silver,bronze,corn,commodities,natural gas,agriculture,emerging market,trade,interest rate,GDP,exports,imports,jobs,tax,tariff,uncertainty,gini,inequality,insurance,pension,liquidity,diversification,alpha,currency,CAPM,fixed income,commission,SEC,yield,real estate,eurodollar,regulations,deficit,yield curve,LIBOR,2008,bubble,energy,1939
0,2006-09-20,23.282972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2006-09-21,23.189838,-0.004008,0.0,-2.0,2.0,0.0,-1.0,8.0,-3.0,-1.0,-34.0,-1.0,-1.0,2.0,0.0,-1.0,-1.0,1.0,0.0,-1.0,0.0,3.0,-1.0,-3.0,-1.0,-6.0,2.0,-11.0,0.0,4.0,-4.0,-2.0,0.0,-7.0,-2.0,0.0,7.0,0.0,0.0,...,-1.0,-5.0,23.0,2.0,-1.0,6.0,8.0,0.0,-2.0,6.0,2.0,8.0,10.0,-1.0,-2.0,-3.0,18.0,4.0,-5.0,3.0,3.0,-8.0,13.0,-4.0,-8.0,14.0,13.0,-7.0,7.0,19.0,0.0,-4.0,-11.0,-13.0,2.0,1.0,0.0,1.0,-3.0,-3.0
2,2006-09-22,23.096708,-0.004024,3.0,-2.0,0.0,0.0,-2.0,-4.0,-2.0,1.0,2.0,1.0,-3.0,-5.0,0.0,-1.0,1.0,-2.0,0.0,-1.0,0.0,-6.0,1.0,-1.0,-9.0,-1.0,-3.0,-11.0,0.0,-8.0,0.0,1.0,1.0,7.0,2.0,-3.0,-9.0,-1.0,-1.0,...,2.0,4.0,-4.0,1.0,11.0,-18.0,-8.0,6.0,0.0,-2.0,0.0,-10.0,-6.0,-1.0,0.0,12.0,-9.0,2.0,-10.0,-8.0,-7.0,1.0,2.0,1.0,-3.0,16.0,-8.0,3.0,-9.0,-20.0,-2.0,-5.0,0.0,4.0,-1.0,-5.0,1.0,11.0,-5.0,0.0
3,2006-09-25,23.282972,0.008032,-3.0,4.0,-1.0,1.0,1.0,-1.0,0.0,1.0,-8.0,0.0,4.0,3.0,0.0,2.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,10.0,0.0,8.0,0.0,6.0,-2.0,2.0,0.0,-2.0,-1.0,-4.0,-2.0,2.0,9.0,0.0,1.0,...,-2.0,-2.0,8.0,-2.0,-7.0,14.0,4.0,-5.0,-4.0,-3.0,7.0,3.0,10.0,2.0,1.0,-9.0,5.0,1.0,11.0,8.0,6.0,10.0,7.0,1.0,7.0,-8.0,13.0,-1.0,6.0,-2.0,8.0,0.0,6.0,14.0,1.0,1.0,0.0,-21.0,6.0,-2.0
4,2006-09-26,23.310913,0.001199,1.0,-2.0,3.0,1.0,3.0,4.0,4.0,0.0,13.0,-1.0,-2.0,2.0,0.0,0.0,-1.0,1.0,-2.0,3.0,-1.0,0.0,0.0,-8.0,6.0,3.0,0.0,5.0,0.0,1.0,4.0,3.0,0.0,1.0,0.0,1.0,3.0,0.0,3.0,...,2.0,-2.0,-25.0,3.0,4.0,-6.0,2.0,3.0,4.0,9.0,-1.0,21.0,-5.0,0.0,0.0,1.0,-7.0,-2.0,5.0,3.0,4.0,-15.0,-27.0,-3.0,-8.0,-17.0,-13.0,3.0,-7.0,7.0,-3.0,9.0,0.0,-21.0,-2.0,5.0,0.0,8.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3770,2021-09-13,24.930000,-0.002804,0.0,0.0,1.0,1.0,3.0,-1.0,3.0,6.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,6.0,0.0,0.0,2.0,-1.0,10.0,1.0,2.0,2.0,6.0,1.0,-1.0,0.0,4.0,-2.0,3.0,1.0,0.0,2.0,...,4.0,1.0,1.0,-1.0,-3.0,3.0,0.0,-2.0,1.0,3.0,1.0,5.0,0.0,1.0,3.0,2.0,2.0,1.0,1.0,2.0,10.0,1.0,0.0,-1.0,0.0,1.0,1.0,0.0,5.0,6.0,0.0,1.0,4.0,3.0,2.0,1.0,0.0,-4.0,5.0,0.0
3771,2021-09-14,24.830000,-0.004019,1.0,0.0,0.0,0.0,-1.0,2.0,0.0,-2.0,-1.0,0.0,1.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,-1.0,-4.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.0,0.0,6.0,1.0,1.0,0.0,2.0,2.0,-1.0,0.0,-1.0,-1.0,...,-1.0,-1.0,1.0,0.0,30.0,5.0,1.0,2.0,-3.0,2.0,-1.0,-7.0,1.0,0.0,3.0,-1.0,-1.0,0.0,0.0,1.0,-2.0,1.0,1.0,-1.0,2.0,2.0,0.0,1.0,0.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,4.0,-1.0
3772,2021-09-15,24.940001,0.004420,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,-7.0,1.0,0.0,0.0,1.0,1.0,...,3.0,1.0,-1.0,0.0,-12.0,0.0,0.0,-1.0,1.0,-1.0,4.0,11.0,-1.0,0.0,0.0,7.0,4.0,1.0,1.0,3.0,-2.0,0.0,-1.0,2.0,7.0,1.0,2.0,0.0,0.0,-4.0,0.0,0.0,0.0,-7.0,-1.0,0.0,0.0,0.0,2.0,3.0
3773,2021-09-16,24.

In [ ]:
ts_df = ts_df.drop(['Adj Close'], axis = 1)
ts_df = ts_df.set_index(ts_df.iloc[:,0])
ts_df = ts_df.drop(['Date'], axis = 1)
ts_df = ts_df.iloc[1:,:]
ts_df

,Log Difference,swiss franc,Europe,EUR,Switzerland,CHF,NASDAQ,S&P500,VIX,volatility,Germany,UK,sterling pound,GBP,Greece,Italy,Spain,France,DAX,unemployment,recession,euro,crisis,fed,finance,financial crisis,federal reserve,gold,inflation,treasuries,OPEC,contagion,corruption,disaster,futures,forwards,president,parliament,aid,EU,...,steel,silver,bronze,corn,commodities,natural gas,agriculture,emerging market,trade,interest rate,GDP,exports,imports,jobs,tax,tariff,uncertainty,gini,inequality,insurance,pension,liquidity,diversification,alpha,currency,CAPM,fixed income,commission,SEC,yield,real estate,eurodollar,regulations,deficit,yield curve,LIBOR,2008,bubble,energy,1939
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2006-09-21,-0.004008,0.0,-2.0,2.0,0.0,-1.0,8.0,-3.0,-1.0,-34.0,-1.0,-1.0,2.0,0.0,-1.0,-1.0,1.0,0.0,-1.0,0.0,3.0,-1.0,-3.0,-1.0,-6.0,2.0,-11.0,0.0,4.0,-4.0,-2.0,0.0,-7.0,-2.0,0.0,7.0,0.0,0.0,0.0,1.0,...,-1.0,-5.0,23.0,2.0,-1.0,6.0,8.0,0.0,-2.0,6.0,2.0,8.0,10.0,-1.0,-2.0,-3.0,18.0,4.0,-5.0,3.0,3.0,-8.0,13.0,-4.0,-8.0,14.0,13.0,-7.0,7.0,19.0,0.0,-4.0,-11.0,-13.0,2.0,1.0,0.0,1.0,-3.0,-3.0
2006-09-22,-0.004024,3.0,-2.0,0.0,0.0,-2.0,-4.0,-2.0,1.0,2.0,1.0,-3.0,-5.0,0.0,-1.0,1.0,-2.0,0.0,-1.0,0.0,-6.0,1.0,-1.0,-9.0,-1.0,-3.0,-11.0,0.0,-8.0,0.0,1.0,1.0,7.0,2.0,-3.0,-9.0,-1.0,-1.0,1.0,0.0,...,2.0,4.0,-4.0,1.0,11.0,-18.0,-8.0,6.0,0.0,-2.0,0.0,-10.0,-6.0,-1.0,0.0,12.0,-9.0,2.0,-10.0,-8.0,-7.0,1.0,2.0,1.0,-3.0,16.0,-8.0,3.0,-9.0,-20.0,-2.0,-5.0,0.0,4.0,-1.0,-5.0,1.0,11.0,-5.0,0.0
2006-09-25,0.008032,-3.0,4.0,-1.0,1.0,1.0,-1.0,0.0,1.0,-8.0,0.0,4.0,3.0,0.0,2.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,10.0,0.0,8.0,0.0,6.0,-2.0,2.0,0.0,-2.0,-1.0,-4.0,-2.0,2.0,9.0,0.0,1.0,0.0,0.0,...,-2.0,-2.0,8.0,-2.0,-7.0,14.0,4.0,-5.0,-4.0,-3.0,7.0,3.0,10.0,2.0,1.0,-9.0,5.0,1.0,11.0,8.0,6.0,10.0,7.0,1.0,7.0,-8.0,13.0,-1.0,6.0,-2.0,8.0,0.0,6.0,14.0,1.0,1.0,0.0,-21.0,6.0,-2.0
2006-09-26,0.001199,1.0,-2.0,3.0,1.0,3.0,4.0,4.0,0.0,13.0,-1.0,-2.0,2.0,0.0,0.0,-1.0,1.0,-2.0,3.0,-1.0,0.0,0.0,-8.0,6.0,3.0,0.0,5.0,0.0,1.0,4.0,3.0,0.0,1.0,0.0,1.0,3.0,0.0,3.0,1.0,0.0,...,2.0,-2.0,-25.0,3.0,4.0,-6.0,2.0,3.0,4.0,9.0,-1.0,21.0,-5.0,0.0,0.0,1.0,-7.0,-2.0,5.0,3.0,4.0,-15.0,-27.0,-3.0,-8.0,-17.0,-13.0,3.0,-7.0,7.0,-3.0,9.0,0.0,-21.0,-2.0,5.0,0.0,8.0,1.0,2.0
2006-09-27,-0.003201,-1.0,6.0,-4.0,-1.0,-5.0,-5.0,-4.0,1.0,-12.0,1.0,0.0,-5.0,0.0,-1.0,0.0,-2.0,0.0,-2.0,0.0,-1.0,0.0,2.0,4.0,-15.0,1.0,2.0,2.0,-1.0,0.0,-2.0,0.0,-7.0,-1.0,-1.0,-11.0,0.0,-3.0,0.0,0.0,...,0.0,-1.0,13.0,-3.0,-9.0,-3.0,-11.0,-2.0,0.0,-12.0,-6.0,-22.0,-2.0,0.0,1.0,-5.0,1.0,-5.0,9.0,-4.0,-3.0,0.0,16.0,0.0,4.0,0.0,-5.0,-1.0,5.0,0.0,-1.0,-9.0,4.0,21.0,1.0,-3.0,0.0,-4.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-13,-0.002804,0.0,0.0,1.0,1.0,3.0,-1.0,3.0,6.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,6.0,0.0,0.0,2.0,-1.0,10.0,1.0,2.0,2.0,6.0,1.0,-1.0,0.0,4.0,-2.0,3.0,1.0,0.0,2.0,2.0,0.0,...,4.0,1.0,1.0,-1.0,-3.0,3.0,0.0,-2.0,1.0,3.0,1.0,5.0,0.0,1.0,3.0,2.0,2.0,1.0,1.0,2.0,10.0,1.0,0.0,-1.0,0.0,1.0,1.0,0.0,5.0,6.0,0.0,1.0,4.0,3.0,2.0,1.0,0.0,-4.0,5.0,0.0
2021-09-14,-0.004019,1.0,0.0,0.0,0.0,-1.0,2.0,0.0,-2.0,-1.0,0.0,1.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,-1.0,-4.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.0,0.0,6.0,1.0,1.0,0.0,2.0,2.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,...,-1.0,-1.0,1.0,0.0,30.0,5.0,1.0,2.0,-3.0,2.0,-1.0,-7.0,1.0,0.0,3.0,-1.0,-1.0,0.0,0.0,1.0,-2.0,1.0,1.0,-1.0,2.0,2.0,0.0,1.0,0.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,4.0,-1.0
2021-09-15,0.004420,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,-7.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,3.0,1.0,-1.0,0.0,-12.0,0.0,0.0,-1.0,1.0,-1.0,4.0,11.0,-1.0,0.0,0.0,7.0,4.0,1.0,1.0,3.0,-2.0,0.0,-1.

In [ ]:
ts_df['Buy_and_Hold'] = (ts_df['Log Difference'] + 1).cumprod()

In [ ]:
trend_port_SD = ts_df['Buy_and_Hold'].std()
trend_port_ER = ts_df['Buy_and_Hold'].iloc[-1] - 1
trend_port_SR = trend_port_ER/trend_port_SD
print(trend_port_SD)
print(trend_port_ER)
print(trend_port_SR)

0.07471772475430048
-0.04933131930508916
-0.6602358338307114



#Buy and Hold Strategy wihtout 0 Consideration

In [ ]:
def trend_strategy(data_frame, trend_name):

  data_frame['Up_Down'] = [1 if i > 0 else -1 for i in data_frame[trend_name]]

  data_frame['Up_Down_Portfolio'] = data_frame['Log Difference'] * data_frame['Up_Down']

  data_frame['Trend_Portfolio_Cum_Ret'] = (data_frame['Up_Down_Portfolio'] + 1).cumprod()

  trend_port_SD = data_frame['Trend_Portfolio_Cum_Ret'].std()

  trend_port_ER = data_frame['Trend_Portfolio_Cum_Ret'].iloc[-1] - 1

  trend_port_SR = trend_port_ER/trend_port_SD

  return [trend_name, trend_port_SD, trend_port_ER, trend_port_SR]

In [ ]:
dfs = []

In [ ]:
for column in ts_df:

    dfs.append(trend_strategy(ts_df, column))

In [ ]:
SD_df = pd.DataFrame(dfs)

In [ ]:
SD_DF = SD_df.iloc[1:155,:]
SD_DF.columns = ['Trends','Standard Deviation','Expected Return','Sharpe Ratio']

In [ ]:
SD_DF_Sort = SD_DF.sort_values(by=['Sharpe Ratio'], ascending=False)
SD_DF_Sort['Pos_Neg'] = [1 if i > 0 else -1 for i in SD_DF_Sort['Sharpe Ratio']]

In [ ]:
import plotly.express as px

fig = px.bar(SD_DF_Sort, x='Trends', y='Sharpe Ratio', color="Pos_Neg", title="Google Trends Long-Short Portfolio")
fig.show()

#Buy and Hold with 0 meaning do nothing

In [ ]:
def trend_strategy1(data_frame, trend_name):

  data_frame['Up_Down'] = [1 if i > 0 else 0 if i == 0 else -1 for i in data_frame[trend_name]]

  data_frame['Up_Down_Portfolio'] = data_frame['Log Difference'] * data_frame['Up_Down']

  data_frame['Trend_Portfolio_Cum_Ret'] = (data_frame['Up_Down_Portfolio'] + 1).cumprod()

  trend_port_SD = data_frame['Trend_Portfolio_Cum_Ret'].std()

  trend_port_ER = data_frame['Trend_Portfolio_Cum_Ret'].iloc[-1] - 1

  trend_port_SR = trend_port_ER/trend_port_SD

  return [trend_name, trend_port_SD, trend_port_ER, trend_port_SR]

In [ ]:
dfs = []

In [ ]:
for column in ts_df:

    dfs.append(trend_strategy1(ts_df, column))

In [ ]:
SD_df = pd.DataFrame(dfs)

In [ ]:
SD_DF = SD_df.iloc[1:155,:]
SD_DF.columns = ['Trends','Standard Deviation','Expected Return','Sharpe Ratio']

In [ ]:
SD_DF_Sort = SD_DF.sort_values(by=['Sharpe Ratio'], ascending=False)
SD_DF_Sort['Pos_Neg'] = [1 if i > 0 else -1 for i in SD_DF_Sort['Sharpe Ratio']]

In [ ]:
import plotly.express as px

fig = px.bar(SD_DF_Sort, x='Trends', y='Sharpe Ratio', color="Pos_Neg", title="Google Trends Long-Short Portfolio")
fig.show()

#Short Only with 0 do Nothing

In [ ]:
ts_df['Up_Down'] = [0 if i >= 0 else -1 for i in ts_df['energy']]
ts_df['Up_Down_Portfolio'] = ts_df['Log Difference'] * ts_df['Up_Down']
ts_df['Trend_Portfolio_Cum_Ret'] = (ts_df['Up_Down_Portfolio'] + 1).cumprod()
trend_port_SD = ts_df['Trend_Portfolio_Cum_Ret'].std()
trend_port_ER = ts_df['Trend_Portfolio_Cum_Ret'].iloc[-1] - 1
trend_port_SR = trend_port_ER/trend_port_SD
print('energy', trend_port_SD, trend_port_ER, trend_port_SR)

energy 0.09209948023204939 0.14929775384311306 1.6210488209808531


In [ ]:
ts_df

,Log Difference,swiss franc,Europe,EUR,Switzerland,CHF,NASDAQ,S&P500,VIX,volatility,Germany,UK,sterling pound,GBP,Greece,Italy,Spain,France,DAX,unemployment,recession,euro,crisis,fed,finance,financial crisis,federal reserve,gold,inflation,treasuries,OPEC,contagion,corruption,disaster,futures,forwards,president,parliament,aid,EU,...,commodities,natural gas,agriculture,emerging market,trade,interest rate,GDP,exports,imports,jobs,tax,tariff,uncertainty,gini,inequality,insurance,pension,liquidity,diversification,alpha,currency,CAPM,fixed income,commission,SEC,yield,real estate,eurodollar,regulations,deficit,yield curve,LIBOR,2008,bubble,energy,1939,Buy_and_Hold,Up_Down,Up_Down_Portfolio,Trend_Portfolio_Cum_Ret
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2006-09-21,-0.004008,0.0,-2.0,2.0,0.0,-1.0,8.0,-3.0,-1.0,-34.0,-1.0,-1.0,2.0,0.0,-1.0,-1.0,1.0,0.0,-1.0,0.0,3.0,-1.0,-3.0,-1.0,-6.0,2.0,-11.0,0.0,4.0,-4.0,-2.0,0.0,-7.0,-2.0,0.0,7.0,0.0,0.0,0.0,1.0,...,-1.0,6.0,8.0,0.0,-2.0,6.0,2.0,8.0,10.0,-1.0,-2.0,-3.0,18.0,4.0,-5.0,3.0,3.0,-8.0,13.0,-4.0,-8.0,14.0,13.0,-7.0,7.0,19.0,0.0,-4.0,-11.0,-13.0,2.0,1.0,0.0,1.0,-3.0,-3.0,0.995992,-1,0.004008,1.004008
2006-09-22,-0.004024,3.0,-2.0,0.0,0.0,-2.0,-4.0,-2.0,1.0,2.0,1.0,-3.0,-5.0,0.0,-1.0,1.0,-2.0,0.0,-1.0,0.0,-6.0,1.0,-1.0,-9.0,-1.0,-3.0,-11.0,0.0,-8.0,0.0,1.0,1.0,7.0,2.0,-3.0,-9.0,-1.0,-1.0,1.0,0.0,...,11.0,-18.0,-8.0,6.0,0.0,-2.0,0.0,-10.0,-6.0,-1.0,0.0,12.0,-9.0,2.0,-10.0,-8.0,-7.0,1.0,2.0,1.0,-3.0,16.0,-8.0,3.0,-9.0,-20.0,-2.0,-5.0,0.0,4.0,-1.0,-5.0,1.0,11.0,-5.0,0.0,0.991984,-1,0.004024,1.008048
2006-09-25,0.008032,-3.0,4.0,-1.0,1.0,1.0,-1.0,0.0,1.0,-8.0,0.0,4.0,3.0,0.0,2.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,10.0,0.0,8.0,0.0,6.0,-2.0,2.0,0.0,-2.0,-1.0,-4.0,-2.0,2.0,9.0,0.0,1.0,0.0,0.0,...,-7.0,14.0,4.0,-5.0,-4.0,-3.0,7.0,3.0,10.0,2.0,1.0,-9.0,5.0,1.0,11.0,8.0,6.0,10.0,7.0,1.0,7.0,-8.0,13.0,-1.0,6.0,-2.0,8.0,0.0,6.0,14.0,1.0,1.0,0.0,-21.0,6.0,-2.0,0.999952,0,0.000000,1.008048
2006-09-26,0.001199,1.0,-2.0,3.0,1.0,3.0,4.0,4.0,0.0,13.0,-1.0,-2.0,2.0,0.0,0.0,-1.0,1.0,-2.0,3.0,-1.0,0.0,0.0,-8.0,6.0,3.0,0.0,5.0,0.0,1.0,4.0,3.0,0.0,1.0,0.0,1.0,3.0,0.0,3.0,1.0,0.0,...,4.0,-6.0,2.0,3.0,4.0,9.0,-1.0,21.0,-5.0,0.0,0.0,1.0,-7.0,-2.0,5.0,3.0,4.0,-15.0,-27.0,-3.0,-8.0,-17.0,-13.0,3.0,-7.0,7.0,-3.0,9.0,0.0,-21.0,-2.0,5.0,0.0,8.0,1.0,2.0,1.001151,0,0.000000,1.008048
2006-09-27,-0.003201,-1.0,6.0,-4.0,-1.0,-5.0,-5.0,-4.0,1.0,-12.0,1.0,0.0,-5.0,0.0,-1.0,0.0,-2.0,0.0,-2.0,0.0,-1.0,0.0,2.0,4.0,-15.0,1.0,2.0,2.0,-1.0,0.0,-2.0,0.0,-7.0,-1.0,-1.0,-11.0,0.0,-3.0,0.0,0.0,...,-9.0,-3.0,-11.0,-2.0,0.0,-12.0,-6.0,-22.0,-2.0,0.0,1.0,-5.0,1.0,-5.0,9.0,-4.0,-3.0,0.0,16.0,0.0,4.0,0.0,-5.0,-1.0,5.0,0.0,-1.0,-9.0,4.0,21.0,1.0,-3.0,0.0,-4.0,0.0,3.0,0.997946,0,-0.000000,1.008048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-13,-0.002804,0.0,0.0,1.0,1.0,3.0,-1.0,3.0,6.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,6.0,0.0,0.0,2.0,-1.0,10.0,1.0,2.0,2.0,6.0,1.0,-1.0,0.0,4.0,-2.0,3.0,1.0,0.0,2.0,2.0,0.0,...,-3.0,3.0,0.0,-2.0,1.0,3.0,1.0,5.0,0.0,1.0,3.0,2.0,2.0,1.0,1.0,2.0,10.0,1.0,0.0,-1.0,0.0,1.0,1.0,0.0,5.0,6.0,0.0,1.0,4.0,3.0,2.0,1.0,0.0,-4.0,5.0,0.0,0.949924,0,-0.000000,1.149759
2021-09-14,-0.004019,1.0,0.0,0.0,0.0,-1.0,2.0,0.0,-2.0,-1.0,0.0,1.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,-1.0,-4.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.0,0.0,6.0,1.0,1.0,0.0,2.0,2.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,...,30.0,5.0,1.0,2.0,-3.0,2.0,-1.0,-7.0,1.0,0.0,3.0,-1.0,-1.0,0.0,0.0,1.0,-2.0,1.0,1.0,-1.0,2.0,2.0,0.0,1.0,0.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,4.0,-1.0,0.946106,0,-0.000000,1.149759
2021-09-15,0.004420,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,-7.0,1.0,0.0,0.0,1

In [ ]:
def trend_short_strategy(data_frame, trend_name):

  data_frame['Up_Down'] = [0 if i >= 0 else -1 for i in data_frame[trend_name]]

  data_frame['Up_Down_Portfolio'] = data_frame['Log Difference'] * data_frame['Up_Down']

  data_frame['Trend_Portfolio_Cum_Ret'] = (data_frame['Up_Down_Portfolio'] + 1).cumprod()

  trend_port_SD = data_frame['Trend_Portfolio_Cum_Ret'].std()

  trend_port_ER = data_frame['Trend_Portfolio_Cum_Ret'].iloc[-1] - 1

  trend_port_SR = trend_port_ER/trend_port_SD

  return [trend_name, trend_port_SD, trend_port_ER, trend_port_SR]

In [ ]:
dfs = []

In [ ]:
for column in ts_df:

    dfs.append(trend_short_strategy(ts_df, column))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning:

invalid value encountered in double_scalars



In [ ]:
SD_df = pd.DataFrame(dfs)

In [ ]:
SD_DF = SD_df.iloc[1:155,:]
SD_DF.columns = ['Trends','Standard Deviation','Expected Return','Sharpe Ratio']

In [ ]:
SD_DF_Sort = SD_DF.sort_values(by=['Sharpe Ratio'], ascending=False)
SD_DF_Sort['Pos_Neg'] = [1 if i > 0 else -1 for i in SD_DF_Sort['Sharpe Ratio']]

In [ ]:
SD_DF_Sort[SD_DF_Sort['Trends']=='energy']

,Trends,Standard Deviation,Expected Return,Sharpe Ratio,Pos_Neg
153,energy,0.092099,0.149298,1.621049,1


In [ ]:
import plotly.express as px

fig = px.bar(SD_DF_Sort, x='Trends', y='Sharpe Ratio', color="Pos_Neg", title="Google Trends Short Portfolio")
fig.show()

#Long Only Approach

In [ ]:
def trend_long_strategy(data_frame, trend_name):

  data_frame['Up_Down'] = [1 if i > 0 else 0 for i in data_frame[trend_name]]

  data_frame['Up_Down_Portfolio'] = data_frame['Log Difference'] * data_frame['Up_Down']

  data_frame['Trend_Portfolio_Cum_Ret'] = (data_frame['Up_Down_Portfolio'] + 1).cumprod()

  trend_port_SD = data_frame['Trend_Portfolio_Cum_Ret'].std()

  trend_port_ER = data_frame['Trend_Portfolio_Cum_Ret'].iloc[-1] - 1

  trend_port_SR = trend_port_ER/trend_port_SD

  return [trend_name, trend_port_SD, trend_port_ER, trend_port_SR]

In [ ]:
dfs = []

In [ ]:
for column in ts_df:

    dfs.append(trend_long_strategy(ts_df, column))

In [ ]:
SD_df = pd.DataFrame(dfs)

In [ ]:
SD_DF = SD_df.iloc[1:155,:]
SD_DF.columns = ['Trends','Standard Deviation','Expected Return','Sharpe Ratio']

In [ ]:
SD_DF_Sort = SD_DF.sort_values(by=['Sharpe Ratio'], ascending=False)
SD_DF_Sort['Pos_Neg'] = [1 if i > 0 else -1 for i in SD_DF_Sort['Sharpe Ratio']]

In [ ]:
import plotly.express as px

fig = px.bar(SD_DF_Sort, x='Trends', y='Sharpe Ratio', color="Pos_Neg", title="Google Trends Short Portfolio")
fig.show()

#Rolling Approach

In [ ]:
ts_df_4500 = ts_df.iloc[0:4500,:]

In [ ]:
df_num_rows = ts_df_4500.shape[0]

In [ ]:
dfs = []

In [ ]:
for i in range(0, df_num_rows, 250):

  print(i)

  ts_df_short = ts_df_4500.iloc[i:i+250,:]

  #print(ts_df_short.head(2))

  for column in ts_df_short:
    dfs.append(trend_strategy(ts_df_short, column))

0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
3000
3250
3500
3750


In [ ]:
chunks = [dfs[x:x+159] for x in range(0, len(dfs), 159)]

In [ ]:
for i in range(0,len(chunks),1):
  p = pd.DataFrame(chunks[i])
  p = p.iloc[1:155,:]
  p.columns = ['Trends','Standard Deviation','Expected Return','Sharpe Ratio']
  p = p.sort_values(by=['Sharpe Ratio'], ascending=False)
  p['Pos_Neg'] = [1 if i > 0 else -1 for i in p['Sharpe Ratio']]

  fig = px.bar(p, x='Trends', y='Sharpe Ratio', color="Pos_Neg", title=f"Google Trends Long-Short Portfolio {2004+i}")
  fig.show()